In [1]:
# Set the work directory to the imaginaire root.
import os, sys, time
import pathlib
root_dir = pathlib.Path().absolute().parents[2]
os.chdir(root_dir)
print(f"Root Directory Path: {root_dir}")

Root Directory Path: /home/cloud/wxz/neuralangelo


In [2]:
# Import Python libraries.
import numpy as np
import torch
import k3d
import json
from collections import OrderedDict
# Import imaginaire modules.
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model

In [3]:
# Read the COLMAP data.
colmap_path = "/home/cloud/datasets/mid_neuralangelo"
cameras, images, points_3D = read_model(path=f"{colmap_path}/dense/sparse", ext=".bin")
# Convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")
# Get the sparse 3D points and the colors.
xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
rgbs = np.stack([point.rgb for point in points_3D.values()])
rgbs = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

# images: 76
# points: 5710


In [7]:
# Visualize the bounding sphere.
json_fname = f"{colmap_path}/dense/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)

center = meta["sphere_center"]
radius = meta["sphere_radius"]
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_center = np.array([0, 0., -1.])
readjust_scale = 1.1
# ------------------------------------------------------------------------------------
center += readjust_center
radius *= readjust_scale
# Make some points to hallucinate a bounding sphere.
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

# --------------------------------------------------------------
ring_radius = 10
ring_points = np.random.randn(10000, 3)
ring_points[:, 1] = 0

ring_points = ring_points / np.linalg.norm(ring_points, axis=-1, keepdims=True)
ring_points = ring_points * ring_radius

# --------------------------------------------------------------
from projects.neuralangelo.data import gen_rot_view
axis = [-1, -1, -1] / np.sqrt(3)
rot_poses = gen_rot_view(0, 360, 10, 60, axis)
rot_poses = torch.tensor(rot_poses[:, :3, :4], dtype=torch.float64)


In [8]:
# Visualize with K3D.
vis_scale = 0.5
# plot = visualize.k3d_visualize_pose(poses,
#                                     vis_depth=(0.5 * vis_scale),
#                                     xyz_length=(0.1 * vis_scale),
#                                     center_size=(0.05 * vis_scale),
#                                     xyz_width=(0.02 * vis_scale))
# print(rot_poses)
plot = visualize.k3d_visualize_pose(rot_poses,
                                    vis_depth=(0.5 * vis_scale),
                                    xyz_length=(0.1 * vis_scale),
                                    center_size=(0.05 * vis_scale),
                                    xyz_width=(0.02 * vis_scale))
plot += k3d.points(xyzs, colors=rgbs, point_size=(0.05 * vis_scale), shader="flat")
plot += k3d.points(sphere_points, color=0x4488ff, point_size=0.02, shader="flat")
# plot += k3d.points(ring_points, color=0xff2400, point_size=0.02, shader="flat")
plot.display()
plot.camera_fov = 30.0

Output()